In [1]:
import os
from ml4floods.data import utils
from tqdm import tqdm

import geopandas as gpd
import numpy as np

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/kikeportales/Documentos/Investigacio/Floods/Credentials/ml4cc-general-access_request_pays.json"
os.environ["GS_USER_PROJECT"] = "ml4cc-general"

fs = utils.get_filesystem("gs://ml4cc_data_lake")


In [2]:
aois = gpd.read_file('gs://ml4cc_data_lake/0_DEV/1_Staging/operational/NEMA001/aois.geojson')
aois

,name,geometry
0,GRID05454,"POLYGON ((144.47329 -36.50516, 144.47329 -36.2..."
1,GRID05455,"POLYGON ((144.47329 -36.30516, 144.47329 -36.0..."
2,GRID05456,"POLYGON ((144.47329 -36.10516, 144.47329 -35.8..."
3,GRID05599,"POLYGON ((144.67329 -36.90516, 144.67329 -36.6..."
4,GRID05600,"POLYGON ((144.67329 -36.70516, 144.67329 -36.4..."
...,...,...
322,GRID11960,"POLYGON ((153.27329 -28.90516, 153.27329 -28.6..."
323,GRID11961,"POLYGON ((153.27329 -28.70516, 153.27329 -28.4..."
324,GRID12106,"POLYGON ((153.47329 -29.10516, 153.47329 -28.8..."
325,GRID12107,"POLYGON ((153.47329 -28.90516, 153.47329 -28.6..."


In [3]:
from datetime import datetime

def get_closest_pre_post_flood(images, flood_date):
    
    flood_date = datetime.strptime(flood_date,'%Y-%m-%d')

    closest_pre = 'none' 
    closest_post = 'none' 
    
    for d in images:
        ims_before = [d for d in images if datetime.strptime(d,'%Y-%m-%d') < flood_date]
        ims_after = [d for d in images if datetime.strptime(d,'%Y-%m-%d') > flood_date]
        
        
        if len(ims_before) > 0:
            closest_pre = min(ims_before, key=lambda x: abs(datetime.strptime(x,'%Y-%m-%d') - flood_date))
            
        if len(ims_after) > 0:
            closest_post = min(ims_after, key=lambda x: abs(datetime.strptime(x,'%Y-%m-%d') - flood_date))
            
    return closest_pre, closest_post

In [4]:
flood_date = '2022-09-23'

pre_ims_s2 = []
post_ims_s2 = []

pre_ims_landsat = []
post_ims_landsat = []

for aoi in tqdm(list(aois.name)):

    ims_s2 = [f.split('/')[-1].split('.tif')[0] for f in fs.glob(f'gs://ml4cc_data_lake/0_DEV/1_Staging/operational/NEMA001/{aoi}/S2/*.tif')]
    closest_before, closest_after = get_closest_pre_post_flood(ims_s2,flood_date) 

    pre_ims_s2.append(closest_before)
    post_ims_s2.append(closest_after)

    ims_landsat = [f.split('/')[-1].split('.tif')[0] for f in fs.glob(f'gs://ml4cc_data_lake/0_DEV/1_Staging/operational/NEMA001/{aoi}/Landsat/*.tif')]
    closest_before, closest_after = get_closest_pre_post_flood(ims_landsat,flood_date) 

    pre_ims_landsat.append(closest_before)
    post_ims_landsat.append(closest_after)



100%|█████████████████████████████████████████| 327/327 [01:49<00:00,  2.99it/s]


In [6]:
aois_stats = aois.copy()
aois_stats['s2_pre'] = pre_ims_s2
aois_stats['s2_post'] = post_ims_s2
aois_stats['landsat_pre'] = pre_ims_landsat
aois_stats['landsat_post'] = post_ims_landsat

aois_stats


,name,geometry,s2_pre,s2_post,landsat_pre,landsat_post
0,GRID05454,"POLYGON ((144.47329 -36.50516, 144.47329 -36.2...",none,none,none,none
1,GRID05455,"POLYGON ((144.47329 -36.30516, 144.47329 -36.0...",none,none,none,none
2,GRID05456,"POLYGON ((144.47329 -36.10516, 144.47329 -35.8...",none,none,none,none
3,GRID05599,"POLYGON ((144.67329 -36.90516, 144.67329 -36.6...",none,none,none,none
4,GRID05600,"POLYGON ((144.67329 -36.70516, 144.67329 -36.4...",none,none,none,none
...,...,...,...,...,...,...
322,GRID11960,"POLYGON ((153.27329 -28.90516, 153.27329 -28.6...",none,none,none,none
323,GRID11961,"POLYGON ((153.27329 -28.70516, 153.27329 -28.4...",none,none,none,none
324,GRID12106,"POLYGON ((153.47329 -29.10516, 153.47329 -28.8...",none,none,none,none
325,GRID12107,"POLYGON ((153.47329 -28.90516, 153.47329 -28.6...",none,none,none,none


In [7]:
aois_stats.explore("s2_pre", tooltip = 'name')


In [8]:
aois_stats.explore("s2_post", tooltip = 'name')